<a href="https://colab.research.google.com/github/mkhetan/ph2_session2/blob/master/EVA_P2S2_File_1_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a reproduction of the IRNN experiment with pixel-by-pixel sequential MNIST in "A Simple Way to Initialize Recurrent Networks of Rectified Linear Units" by Quoc V. Le, Navdeep Jaitly, Geoffrey E. Hinton

arxiv:1504.00941v2 [cs.NE] 7 Apr 2015 http://arxiv.org/pdf/1504.00941v2.pdf

Optimizer is replaced with RMSprop which yields more stable and steady improvement.

Reaches 0.93 train/test accuracy after 900 epochs (which roughly corresponds to 1687500 steps in the original paper.)

In [0]:
%tensorflow_version 1.x
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN
from keras import initializers
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [0]:
#batch_size = 1024
# experimenting batch size of 784...
batch_size = 100

num_classes = 10
epochs = 10
hidden_units = 784

learning_rate = 1e-6
clip_norm = 1.0

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


The original code was feeding the data 1 pixel at a time. Changed that too 1 digit ( (1,784) matrix at a time )
Also - gave batch size 100 - so 100 digits will be fed in 1 shot

In [0]:
#x_train = x_train.reshape(x_train.shape[0], -1, 1)
#x_test = x_test.reshape(x_test.shape[0], -1, 1)

# instead of x_train being 60000,784,1 - we are making it 60000,1,784 so that we can feed 1 row (1 digit) in one shot
x_train = x_train.reshape(x_train.shape[0], 1, -1)
x_test = x_test.reshape(x_test.shape[0], 1, -1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(x_train.shape[1:])

x_train shape: (60000, 1, 784)
60000 train samples
10000 test samples
(1, 784)


THe input here is 1 pixel at a time!!! That's too much!


In [0]:
print('Evaluate IRNN...')
model = Sequential()
model.add(SimpleRNN(hidden_units, activation='relu', input_shape=x_train.shape[1:]))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])
model.summary()

Evaluate IRNN...





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 784)               1230096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                7850      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 1,237,946
Trainable params: 1,237,946
Non-trainable params: 0
_________________________________________________________________


This model is achieving the following:
IRNN test score: 2.302489097213745
IRNN test accuracy: 0.1201

28x28 image data is flattened into 784 x 1 - this is one row that is fed is input.
But, there are only 100 units in the RNN... 
RNN output is 100 which gets added to input 784 - so input will be 884 - and only 100 units of RNN - could this be a problem?


In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

scores = model.evaluate(x_test, y_test, verbose=0)
print('IRNN test score:', scores[0])
print('IRNN test accuracy:', scores[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/10





60000/60000 [==============================] - 4s 73us/step - loss: 2.2761 - acc: 0.1205 - val_loss: 2.1799 - val_acc: 0.1862
Epoch 2/10
60000/60000 [==============================] - 3s 51us/step - loss: 2.0986 - acc: 0.3056 - val_loss: 2.0093 - val_acc: 0.4381
Epoch 3/10
60000/60000 [==============================] - 3s 50us/step - loss: 1.9392 - acc: 0.5174 - val_loss: 1.8537 - val_acc: 0.6098
Epoch 4/10
60000/60000 [==============================] - 3s 51us/step - loss: 1.7928 - acc: 0.6373 - val_loss: 1.7103 - val_acc: 0.6935
Epoch 5/10
60000/60000 [==============================] - 3s 52us/step - loss: 1.6573 - acc: 0.7035 - val_loss: 1.5774 - val_acc: 0.7417
Epoch 6/10
60000/60000 [==============================] - 3s 50us/step - loss: 1.5323 - acc: 0.7430 - val_loss: 1.4553 - val_acc: 0.7721
Epoch 7/10
60000/60000 [==